<a href="https://colab.research.google.com/github/Sarvesh1814/US-Book-Recommendation-System/blob/main/User_based_Colabrative_for_BOOK_Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [112]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [113]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

## Loading Data

In [114]:
df = pd.read_csv("/content/drive/MyDrive/RS Data/Assignment 3/Processed_data/All_User_Book_Interactions")
df.head(5)

,User-ID,ISBN,Book-Rating,Book-Title
0,277157,0671024094,7,The Coffin Dancer (Lincoln Rhyme Novels (Paper...
1,277195,0060391626,10,I Know This Much Is True (Oprah's Book Club)
2,277195,0671021001,10,She's Come Undone (Oprah's Book Club)
3,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...
4,277427,0061009059,9,One for the Money (Stephanie Plum Novels (Pape...


In [115]:
df.shape

(29920, 4)

In [141]:
df = df[pd.to_numeric(df['ISBN'], errors='coerce').notnull()]

In [142]:
df.shape

(27071, 4)

In [143]:
df = df[pd.to_numeric(df['User-ID'], errors='coerce').notnull()]

In [144]:
df.shape

(27071, 4)

In [145]:
len(df["User-ID"].unique())

2421

In [146]:
len(df["ISBN"].unique())

965

## Train-Test Splitting

In [147]:
df.head()

,User-ID,ISBN,Book-Rating,Book-Title
0,277157,0671024094,7,The Coffin Dancer (Lincoln Rhyme Novels (Paper...
1,277195,0060391626,10,I Know This Much Is True (Oprah's Book Club)
2,277195,0671021001,10,She's Come Undone (Oprah's Book Club)
4,277427,0061009059,9,One for the Money (Stephanie Plum Novels (Pape...
5,277427,0316776963,8,Me Talk Pretty One Day


In [148]:
def split_group(group, split_ratio=0.8):
    n_total = len(group)
    n_train = int(n_total * split_ratio)
    idx = group.index
    train_idx = idx[:n_train].tolist()
    test_idx = idx[n_train:].tolist()
    return train_idx, test_idx

In [149]:
grouped = df.groupby('User-ID')
indices = grouped.apply(split_group)

# concatenate the train and test indices for each group
train_idx = pd.Index([idx for indices in indices for idx in indices[0]])
test_idx = pd.Index([idx for indices in indices for idx in indices[1]])

# create the train and test dataframes
train_df = df.loc[train_idx]
test_df = df.loc[test_idx]

In [150]:
train_df.head()

,User-ID,ISBN,Book-Rating,Book-Title
111,243,0060915544,10,The Bean Trees
112,243,0060977493,7,The God of Small Things
113,243,0316601950,9,The Pilot's Wife : A Novel
114,243,0316776963,9,Me Talk Pretty One Day
115,243,0316899984,7,"River, Cross My Heart"


In [151]:
train_df.shape

(20644, 4)

In [152]:
test_df.head()

,User-ID,ISBN,Book-Rating,Book-Title
120,243,0446606383,6,The Midnight Club
121,243,0449006522,6,Manhattan Hunt Club
133,254,0451526341,8,Animal Farm
134,254,0590353403,9,Harry Potter and the Sorcerer's Stone (Book 1)
135,254,0671021001,7,She's Come Undone (Oprah's Book Club)


In [153]:
test_df.shape

(6427, 4)

In [154]:
unique_users = df['User-ID'].unique() 
unique_books = df['ISBN'].unique()

In [155]:
users_dict = {unique_users[i] : i for i in range(len(unique_users))}
books_dict = {unique_books[i] : i for i in range(len(unique_books))}

In [156]:
users_dict_rev = {int(v): k for k, v in users_dict.items()}
books_dict_rev = {int(v): k for k, v in books_dict.items()}

In [157]:
train_df['User-ID'] = train_df['User-ID'].map(users_dict)
train_df['ISBN'] = train_df['ISBN'].map(books_dict)
test_df['User-ID'] = test_df['User-ID'].map(users_dict)
test_df['ISBN'] = test_df['ISBN'].map(books_dict)

## Creating Pivot Table

In [158]:
train_pt = train_df.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating')
train_pt_new = train_pt.copy()
test_pt = test_df.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating')

In [187]:
test_pt

ISBN,0,2,7,9,10,12,13,14,18,19,...,949,950,951,952,953,954,957,959,960,961
User-ID,,,,,,,,,,,,,,,,,,,,,
0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2416,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2417,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2418,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Creating Sparse Matrix

In [161]:
import scipy.sparse as sp

train_sparse = sp.csr_matrix(train_pt.fillna(0).values)
test_sparse = sp.csr_matrix(test_pt.fillna(0).values)


In [162]:
train_dense = train_sparse.toarray()
test_dense = test_sparse.toarray()
train_dense.shape

(2293, 928)

In [163]:
np.count_nonzero(train_dense[0]),np.count_nonzero(test_dense[0])

(1, 1)

## Filling missing Values based on User-User Affinity score
( We are taking 25 Neighbours in consideration because any more and it is taking too much time to calculate)


### Applying KNN Model

In [164]:
from sklearn.neighbors import NearestNeighbors

knn_model = NearestNeighbors(metric='cosine', algorithm='brute',n_neighbors=25)
knn_model.fit(train_sparse)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=25)

In [165]:
distances, indices = knn_model.kneighbors(train_dense, n_neighbors=25)

In [166]:
indices

array([[   0, 1640,  520, ..., 1694, 1867,  716],
       [   1, 1836,  464, ..., 1462,  397, 2242],
       [   2,  479,  847, ...,  850,  273, 2223],
       ...,
       [2290, 2020, 1401, ...,  149,  672,  932],
       [2291,  743, 1279, ..., 2052, 1431,  364],
       [2292, 1424, 1738, ...,  908, 1401,  201]])

In [167]:
distances

array([[0.00000000e+00, 2.44071054e-01, 5.15498417e-01, ...,
        8.09201257e-01, 8.45286813e-01, 8.64646350e-01],
       [0.00000000e+00, 6.52669409e-01, 7.25214081e-01, ...,
        8.16058198e-01, 8.16720911e-01, 8.17478619e-01],
       [1.11022302e-16, 5.04269933e-01, 5.69337238e-01, ...,
        7.49747743e-01, 7.52805454e-01, 7.60414811e-01],
       ...,
       [0.00000000e+00, 5.34757895e-01, 5.34757895e-01, ...,
        7.43104343e-01, 7.45811084e-01, 7.52420444e-01],
       [0.00000000e+00, 6.24220311e-01, 6.94215489e-01, ...,
        8.22245122e-01, 8.23467655e-01, 8.24672668e-01],
       [1.11022302e-16, 6.30537389e-01, 6.51355039e-01, ...,
        7.49510284e-01, 7.54950985e-01, 7.54950985e-01]])

In [168]:
dists = cosine_similarity(train_sparse)
inds = dists.argsort()[:, ::-1]

## Recommendation for single User

In [175]:
from scipy.sparse import find
from sklearn.metrics.pairwise import cosine_similarity

def get_top_k_recommendations(User_id, k, train_sparse, users_dict, dists = None, inds = None):
    user_id = users_dict[User_id]
 
    original_ratings = train_sparse.getrow(user_id).toarray()[0]
    
    # Create a mask for books that have not been rated by the user
    mask = original_ratings == 0
    
    # Fill missing values and select only the unrated books
    train_sparse_new = fill_missing_values(user_id, train_sparse,dists, inds)
    recommendations = train_sparse_new.getrow(user_id).toarray()[0, mask]
    
    # Sort the recommended books in descending order by predicted rating
    top_k = (-recommendations).argsort()[:k]
    top_k_books_ids = [books_dict_rev[x] for x in top_k]
    return top_k_books_ids

def predict_rating(user_id, books_id, train_sparse, dists = None, inds = None):
    # Get the indices of the K nearest neighbors for the user
    dist = dists[user_id]
    ind = inds[user_id]
    
    # Calculate the weighted average of the ratings for the K nearest neighbors
    neighbor_ratings = train_sparse.getcol(books_id).toarray().ravel()[ind]
    neighbor_distances = 1 - dist
    
    mask = ~np.isnan(neighbor_ratings)
    numerator = np.sum(neighbor_ratings[mask] * neighbor_distances[mask])
    denominator = np.sum(neighbor_distances[mask])
    
    if denominator == 0:
        return None
    else:
        return numerator / denominator

def fill_missing_values(user_id, train_sparse, dists = None, inds = None):
    train_sparse_new = train_sparse.copy()
    if dists is None or inds is None:
        # Compute distances and indices of nearest neighbors for all users
        dists = cosine_similarity(train_sparse)
        inds = dists.argsort()[:, ::-1]
    # Find unrated books for the user
    row, col, _ = find(train_sparse.getrow(user_id))
    unrated_books_ids = set(range(train_sparse.shape[1])) - set(col)
    
    # Fill in missing values for each unrated books
    for books_id in unrated_books_ids:
        predicted_rating = predict_rating(user_id, books_id, train_sparse, dists, inds)
        if predicted_rating is not None:
            train_sparse_new[user_id, books_id] = predicted_rating
    
    return train_sparse_new


In [189]:
top_books = get_top_k_recommendations(243	,10, train_sparse, users_dict, dists, inds)
top_books

/usr/local/lib/python3.10/dist-packages/scipy/sparse/_index.py:103: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


['0060987561',
 '0385504209',
 '0425178579',
 '0385505833',
 '0743206045',
 '0553582526',
 '0671617028',
 '0684814994',
 '0446523569',
 '0440211727']

In [178]:
recipe_names = df.loc[df['ISBN'].isin(top_books),'Book-Title'].unique()
for element in recipe_names:
    print(element)

The Da Vinci Code
A Time to Kill
Daddy's Little Girl
I Know This Much Is True
Christmas Box (Christmas Box Trilogy)
Skipping Christmas
Message in a Bottle
The Color Purple
Dust to Dust
Betrayal in Death


## Evaluation

In [190]:
def evaluate_recommendations(user_id, k,dists,inds):
    # get the actual rated books for the user
    user_ID = users_dict[user_id]
    actual_rated_books = test_pt.loc[user_ID]
    actual_rated_books = actual_rated_books[actual_rated_books.notna()]
    actual_rated_books_set = set(actual_rated_books.index)
    
    # get the top-k recommended books for the user
    top_k_recommendations = get_top_k_recommendations(user_id, k,train_sparse,users_dict,dists,inds)
    top_k_recommendations_set = set(top_k_recommendations)
    
    # calculate precision and recall
    intersection = actual_rated_books_set.intersection(top_k_recommendations_set)
    if len(top_k_recommendations_set) == 0:
      precision = 0.0
    else:
      precision = len(intersection) / len(top_k_recommendations_set)
    recall = len(intersection) / len(actual_rated_books_set)
    
    return precision, recall

In [191]:
P,R = evaluate_recommendations(243,10,dists,inds)
print("Precision: ",P)
print("Recall: ",R)

Precision:  0.0
Recall:  0.0


In [ ]:
k_values = [5, 10, 50, 60,70,80,90, 100]
precision_scores = []
recall_scores = []

for k in k_values:
    total_precision = 0
    total_recall = 0
    num_users = 0

    for user_id in test_pt.columns:
        if not test_pt[user_id].isnull().all():
            while(user_id == 0):
              user_id = user_id+1
            precision, recall = evaluate_recommendations(users_dict_rev[user_id], k,dists,inds)
            total_precision += precision
            total_recall += recall
            num_users += 1

    precision_scores.append(total_precision / num_users)
    recall_scores.append(total_recall / num_users)

    print("Precision scores:", precision_scores)
    print("Recall scores:", recall_scores)

/usr/local/lib/python3.10/dist-packages/scipy/sparse/_index.py:103: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


In [ ]:
import matplotlib.pyplot as plt

plt.plot(k_values, precision_scores, label='Precision')
plt.plot(k_values, recall_scores, label='Recall')
plt.xlabel('K')
plt.ylabel('Score')
plt.legend()
plt.show()